<a href="https://colab.research.google.com/github/visshal2301/AdvanceSpark_GoogleColab/blob/main/Shubham_2_Join_Partition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Spark Session
from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    .appName("Joins and Data Partitions")
    .master("local[*]")
    .getOrCreate()
)

spark

In [ ]:
# Emp Data & Schema

emp_data = [
    ["001","101","John Doe","30","Male","50000","2015-01-01"],
    ["002","101","Jane Smith","25","Female","45000","2016-02-15"],
    ["003","102","Bob Brown","35","Male","55000","2014-05-01"],
    ["004","102","Alice Lee","28","Female","48000","2017-09-30"],
    ["005","103","Jack Chan","40","Male","60000","2013-04-01"],
    ["006","103","Jill Wong","32","Female","52000","2018-07-01"],
    ["007","101","James Johnson","42","Male","70000","2012-03-15"],
    ["008","102","Kate Kim","29","Female","51000","2019-10-01"],
    ["009","103","Tom Tan","33","Male","58000","2016-06-01"],
    ["010","104","Lisa Lee","27","Female","47000","2018-08-01"],
    ["011","104","David Park","38","Male","65000","2015-11-01"],
    ["012","105","Susan Chen","31","Female","54000","2017-02-15"],
    ["013","106","Brian Kim","45","Male","75000","2011-07-01"],
    ["014","107","Emily Lee","26","Female","46000","2019-01-01"],
    ["015","106","Michael Lee","37","Male","63000","2014-09-30"],
    ["016","107","Kelly Zhang","30","Female","49000","2018-04-01"],
    ["017","105","George Wang","34","Male","57000","2016-03-15"],
    ["018","104","Nancy Liu","29","","50000","2017-06-01"],
    ["019","103","Steven Chen","36","Male","62000","2015-08-01"],
    ["020","102","Grace Kim","32","Female","53000","2018-11-01"]
]

emp_schema = "employee_id string, department_id string, name string, age string, gender string, salary string, hire_date string"

dept_data = [
    ["101", "Sales", "NYC", "US", "1000000"],
    ["102", "Marketing", "LA", "US", "900000"],
    ["103", "Finance", "London", "UK", "1200000"],
    ["104", "Engineering", "Beijing", "China", "1500000"],
    ["105", "Human Resources", "Tokyo", "Japan", "800000"],
    ["106", "Research and Development", "Perth", "Australia", "1100000"],
    ["107", "Customer Service", "Sydney", "Australia", "950000"]
]

dept_schema = "department_id string, department_name string, city string, country string, budget string"

In [ ]:
emp = spark.createDataFrame(data=emp_data, schema=emp_schema)
dept = spark.createDataFrame(data=dept_data, schema=dept_schema)

In [ ]:
# Show emp dataframe (ACTION)

emp.show()
dept.show()

+-----------+-------------+-------------+---+------+------+----------+
|employee_id|department_id|         name|age|gender|salary| hire_date|
+-----------+-------------+-------------+---+------+------+----------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|
|        006|          103|    Jill Wong| 32|Female| 52000|2018-07-01|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|
|        009|          103|      Tom Tan| 33|  Male| 58000|2016-06-01|
|        010|          104|     Lisa Lee| 27|Female| 47000|2018-08-01|
|        011|          104|   David Park| 38|  Male| 65000|2015-11-01|
|     

In [ ]:
# Print Schema

emp.printSchema()
dept.printSchema()

root
 |-- employee_id: string (nullable = true)
 |-- department_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- hire_date: string (nullable = true)

root
 |-- department_id: string (nullable = true)
 |-- department_name: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- budget: string (nullable = true)



In [ ]:
emp.rdd.getNumPartitions()

2

In [ ]:
# Get number of partitions for dept
dept.rdd.getNumPartitions()

2

In [ ]:
#repartition
emp=emp.repartition(8)

In [ ]:
emp.rdd.getNumPartitions()

8

In [ ]:
#partition on basis of some column
emp_partitioned = emp.repartition(4, "department_id")

In [ ]:
emp_partitioned.rdd.getNumPartitions()

4

In [ ]:
from pyspark.sql.functions import spark_partition_id

emp_1 = emp.withColumn("partition_num",spark_partition_id())
emp_1.show()

+-----------+-------------+-------------+---+------+------+----------+-------------+
|employee_id|department_id|         name|age|gender|salary| hire_date|partition_num|
+-----------+-------------+-------------+---+------+------+----------+-------------+
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|            0|
|        016|          107|  Kelly Zhang| 30|Female| 49000|2018-04-01|            0|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|            1|
|        011|          104|   David Park| 38|  Male| 65000|2015-11-01|            1|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|            2|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|            2|
|        020|          102|    Grace Kim| 32|Female| 53000|2018-11-01|            2|
|        019|          103|  Steven Chen| 36|  Male| 62000|2015-08-01|            2|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-0

In [ ]:
from pyspark.sql.functions import spark_partition_id
#repartition based on some column
emp_1 = emp.repartition(4,"department_id").withColumn("partition_num",spark_partition_id())
emp_1.show()

+-----------+-------------+-------------+---+------+------+----------+-------------+
|employee_id|department_id|         name|age|gender|salary| hire_date|partition_num|
+-----------+-------------+-------------+---+------+------+----------+-------------+
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|            0|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|            0|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|            0|
|        014|          107|    Emily Lee| 26|Female| 46000|2019-01-01|            0|
|        016|          107|  Kelly Zhang| 30|Female| 49000|2018-04-01|            0|
|        020|          102|    Grace Kim| 32|Female| 53000|2018-11-01|            0|
|        012|          105|   Susan Chen| 31|Female| 54000|2017-02-15|            1|
|        017|          105|  George Wang| 34|  Male| 57000|2016-03-15|            1|
|        010|          104|     Lisa Lee| 27|Female| 47000|2018-0

In [ ]:
#join
# INNER JOIN datasets
# select e.emp_name, d.department_name, d.department_id, e.salary
# from emp e inner join dept d on emp.department_id = dept.department_id

df_joined = emp.alias("e").join(dept.alias("d"), how="inner", on=emp.department_id==dept.department_id)
df_joined.select("e.name","d.department_name","d.department_id","e.salary").show()


+-------------+--------------------+-------------+------+
|         name|     department_name|department_id|salary|
+-------------+--------------------+-------------+------+
|James Johnson|               Sales|          101| 70000|
|   Jane Smith|               Sales|          101| 45000|
|     John Doe|               Sales|          101| 50000|
|    Grace Kim|           Marketing|          102| 53000|
|    Alice Lee|           Marketing|          102| 48000|
|     Kate Kim|           Marketing|          102| 51000|
|    Bob Brown|           Marketing|          102| 55000|
|      Tom Tan|             Finance|          103| 58000|
|  Steven Chen|             Finance|          103| 62000|
|    Jill Wong|             Finance|          103| 52000|
|    Jack Chan|             Finance|          103| 60000|
|  Kelly Zhang|    Customer Service|          107| 49000|
|    Emily Lee|    Customer Service|          107| 46000|
|    Nancy Liu|         Engineering|          104| 50000|
|     Lisa Lee

In [ ]:
# LEFT OUTER JOIN datasets
# select e.emp_name, d.department_name, d.department_id, e.salary
# from emp e left outer join dept d on emp.department_id = dept.department_id

df_joined = emp.alias("e").join(dept.alias("d"), how="left_outer", on=emp.department_id==dept.department_id)

In [ ]:
df_joined.select("e.name", "d.department_name", "d.department_id", "e.salary").show()

+-------------+--------------------+-------------+------+
|         name|     department_name|department_id|salary|
+-------------+--------------------+-------------+------+
|    Bob Brown|           Marketing|          102| 55000|
|    Emily Lee|    Customer Service|          107| 46000|
|    Jack Chan|             Finance|          103| 60000|
|   David Park|         Engineering|          104| 65000|
|    Jill Wong|             Finance|          103| 52000|
|     Kate Kim|           Marketing|          102| 51000|
|  George Wang|     Human Resources|          105| 57000|
|  Michael Lee|Research and Deve...|          106| 63000|
|     Lisa Lee|         Engineering|          104| 47000|
|    Alice Lee|           Marketing|          102| 48000|
|  Steven Chen|             Finance|          103| 62000|
|    Nancy Liu|         Engineering|          104| 50000|
|     John Doe|               Sales|          101| 50000|
|    Brian Kim|Research and Deve...|          106| 75000|
|      Tom Tan

In [ ]:
# Bonus TIP
# Joins with cascading conditions
# Join with Department_id and only for departments 101 or 102
# Join with not null/null conditions

df_final = emp.join(dept, how="left_outer",
                   on=(emp.department_id==dept.department_id) & ((emp.department_id == "101") | (emp.department_id == "102"))
                    & (emp.salary.isNull())
                   )

In [ ]:
df_final.show()

+-----------+-------------+-------------+---+------+------+----------+-------------+---------------+----+-------+------+
|employee_id|department_id|         name|age|gender|salary| hire_date|department_id|department_name|city|country|budget|
+-----------+-------------+-------------+---+------+------+----------+-------------+---------------+----+-------+------+
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|         NULL|           NULL|NULL|   NULL|  NULL|
|        016|          107|  Kelly Zhang| 30|Female| 49000|2018-04-01|         NULL|           NULL|NULL|   NULL|  NULL|
|        011|          104|   David Park| 38|  Male| 65000|2015-11-01|         NULL|           NULL|NULL|   NULL|  NULL|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|         NULL|           NULL|NULL|   NULL|  NULL|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|         NULL|           NULL|NULL|   NULL|  NULL|
|        020|          102|    G